In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import requests

url = "https://dhakahouserents.s3.ap-southeast-1.amazonaws.com/bproperties_dhaka_house_rents.zip"
data = requests.get(url)
data

<Response [200]>

In [4]:
import zipfile
import os

if data.status_code == 200:  # Check for successful download
    os.makedirs("data_ingestion", exist_ok = True)
    
    with open("data.zip", "wb") as f:
        f.write(data.content)

    with zipfile.ZipFile("data.zip", 'r') as zip_ref:
        zip_ref.extractall("data_ingestion")  # Extract all files from the ZIP archive

    print("ZIP file downloaded and extracted successfully!")
else:
    print(f"Download failed. Status code: {data.status_code}")


ZIP file downloaded and extracted successfully!


In [5]:
df = pd.read_csv("data_ingestion/houserentdhaka.csv")

In [6]:
df.shape

(28800, 6)

In [7]:
df.head(3)

,Unnamed: 0,Location,Area,Bed,Bath,Price
0,0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand
1,1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand
2,2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand


In [8]:
df.drop("Unnamed: 0", axis=1, inplace = True)

In [9]:
df.head()

,Location,Area,Bed,Bath,Price
0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand
1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand
2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand
3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,75 Thousand
4,"Baridhara, Dhaka","2,200 sqft",3,3,75 Thousand


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Location  28800 non-null  object
 1   Area      28800 non-null  object
 2   Bed       28800 non-null  int64 
 3   Bath      28800 non-null  int64 
 4   Price     28800 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


## Pre-Process the Location Column

In [11]:
def preprocess_location(address):
    address = address.lower()
    location_arr = address.split(', ')
    location = location_arr[-2] if len(location_arr)> 1 else None
    return location 

df["location"] = df["Location"].apply(preprocess_location)


In [12]:
df["location"].unique()

array(['bashundhara r-a', 'tejgaon', 'mohammadpur', 'gulshan',
       'baridhara', 'hazaribag', 'mirpur', 'nikunja', 'uttara',
       'khilgaon', 'ibrahimpur', 'badda', 'adabor', 'jatra bari',
       'malibagh', 'banani', 'kakrail', 'dhanmondi', 'maghbazar',
       'kalachandpur', 'niketan', 'eskaton', 'banasree', 'bashabo',
       'baridhara dohs', 'aftab nagar', 'lalmatia', 'dakshin khan',
       'mohakhali dohs', 'sutrapur', 'hatirpool', 'agargaon', 'rampura',
       'cantonment', 'shahbagh', 'khilkhet', 'motijheel', 'shantinagar',
       'shegunbagicha', 'kathalbagan', 'shyamoli', 'kalabagan', 'demra',
       'kuril', 'mohakhali', 'lalbagh', 'new market', 'kafrul',
       'kachukhet', 'turag', None, 'nadda', 'shyampur', 'maniknagar',
       'banani dohs', 'shiddheswari', 'bangshal', 'paribagh',
       'joar sahara', 'mugdapara', 'north shahjahanpur', 'kotwali',
       'shahjahanpur', 'uttar khan', 'taltola', 'sadarghat',
       'banglamotors', 'zafrabad', 'keraniganj'], dtype=objec

In [13]:
df["location"].isnull().sum()

13

## Pre-Process the Area Column

In [14]:
df['Area']

0        1,600 sqft
1          900 sqft
2        1,250 sqft
3        2,200 sqft
4        2,200 sqft
            ...    
28795    1,400 sqft
28796    1,400 sqft
28797    1,250 sqft
28798    1,900 sqft
28799    1,150 sqft
Name: Area, Length: 28800, dtype: object

In [15]:
import re
def preprocess_area(text):
#     text = text.strip()
    text = text.replace(',', '')
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())

In [16]:
df['area_sqft'] = df['Area'].apply(preprocess_area)

In [17]:
df['area_sqft']

0        1600
1         900
2        1250
3        2200
4        2200
         ... 
28795    1400
28796    1400
28797    1250
28798    1900
28799    1150
Name: area_sqft, Length: 28800, dtype: int64

In [18]:
df['area_sqft'].unique()

array([1600,  900, 1250, 2200, 3000, 2300,  950, 1150, 1300,  750, 1850,
       1200, 1050, 1400, 2100,  800, 1500,  650, 1524, 1100, 3500,  850,
       1750, 1617, 1350, 1900, 2600, 1860, 3200, 1895, 1360, 1000, 1800,
       2150, 2000, 3680, 1058, 2420, 1650, 2700, 2660, 1570, 3245,  745,
       2400,  600, 1180, 2550, 2025, 1450, 2359, 2021, 2960,  720, 1550,
        700, 2750,  500, 4200, 1057, 2250,  400, 1385, 2800, 1700,  550,
       1612, 1890, 1760, 5500, 1393, 2500,  870, 1665, 1530, 3750, 1320,
       1372, 2450, 2720, 2840, 1778, 1776, 1410, 4500, 1663, 1950, 4700,
       2015, 1943, 1710, 3300, 1215, 2050, 1480, 3100, 1165, 2075, 1875,
       1075, 4000,  970, 1765, 3800, 1810,  980, 1580, 2045, 1735, 2900,
       2350, 1830, 1654,  880, 3346, 1270, 1124, 5000, 1980, 1625, 1170,
       1730, 1485,  760, 2091,  725, 3950, 2201, 3600, 1975, 1660, 5800,
       1675, 2580, 1240, 1442, 1375, 1465, 6300, 2146, 1888,  200, 4300,
       2775, 3244,  780, 1020,  920, 1285, 1325, 13

## Pre-Process the Price Column

In [19]:
df['Price'].unique()

array(['20 Thousand', '18 Thousand', '75 Thousand', '50 Thousand',
       '14 Thousand', '28 Thousand', '19 Thousand', '40 Thousand',
       '60 Thousand', '16 Thousand', '35 Thousand', '1.6 Lakh',
       '25 Thousand', '30 Thousand', '18.63 Thousand', '32 Thousand',
       '1 Lakh', '2 Lakh', '10 Thousand', '15 Thousand', '27 Thousand',
       '24 Thousand', '38 Thousand', '80 Thousand', '1.5 Lakh',
       '1.2 Lakh', '26 Thousand', '45 Thousand', '85 Thousand',
       '13 Thousand', '1.1 Lakh', '24.76 Thousand', '17 Thousand',
       '95 Thousand', '55 Thousand', '31 Thousand', '2.4 Lakh',
       '54 Thousand', '90 Thousand', '42 Thousand', '1.4 Lakh',
       '33 Thousand', '8.5 Thousand', '11 Thousand', '12 Thousand',
       '4 Lakh', '65 Thousand', '36 Thousand', '70 Thousand',
       '52.5 Thousand', '2.5 Lakh', '5 Thousand', '3.5 Lakh', '1.15 Lakh',
       '15.5 Thousand', '36.75 Thousand', '33.5 Thousand', '3 Lakh',
       '22 Thousand', '12.5 Thousand', '2.15 Lakh', '7 Thousand

In [20]:
def extract_price(price_str):
    text = str(price_str).strip()
    text = text.replace(',', '')
    # Define conversion factors
    conversion_factors = {
        'Thousand': 1000,
        'Lakh': 100000,
        'Cr': 10000000
    }
    
    # Check for 'Lac' and 'Cr' and handle them
    for unit, factor in conversion_factors.items():
        if unit in text:
            match = re.search(r'\d+(\.\d+)?', text)
            if match:
                return float(match.group()) * factor
    
    # Handle the case with pure numbers and numbers followed by other text
    match = re.search(r'\d+(\.\d+)?', text)
    if match:
        return int(match.group())
    
    return None

In [21]:
df['avg_rent'] = df['Price'].apply(extract_price)

In [22]:
df['avg_rent'].unique()

array([ 20000.,  18000.,  75000.,  50000.,  14000.,  28000.,  19000.,
        40000.,  60000.,  16000.,  35000., 160000.,  25000.,  30000.,
        18630.,  32000., 100000., 200000.,  10000.,  15000.,  27000.,
        24000.,  38000.,  80000., 150000., 120000.,  26000.,  45000.,
        85000.,  13000., 110000.,  24760.,  17000.,  95000.,  55000.,
        31000., 240000.,  54000.,  90000.,  42000., 140000.,  33000.,
         8500.,  11000.,  12000., 400000.,  65000.,  36000.,  70000.,
        52500., 250000.,   5000., 350000., 115000.,  15500.,  36750.,
        33500., 300000.,  22000.,  12500., 215000.,   7000.,  13500.,
       280000.,  39000.,  11500.,   6500., 135000., 130000.,  21000.,
        45500.,  16530.,  17500.,   7500.,  57000.,  19260.,  67500.,
        23000.,   9000.,  16500.,  34000.,  37000.,  28500.,  48000.,
        18500.,  24500.,  34500., 125000., 170000.,  14500., 650000.,
        10500.,  52000.,  37500.,   8000.,  17990., 180000., 185000.,
       270000.,  182

In [23]:
df.head(3)

,Location,Area,Bed,Bath,Price,location,area_sqft,avg_rent
0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand,bashundhara r-a,1600,20000.0
1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand,tejgaon,900,20000.0
2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand,mohammadpur,1250,18000.0


In [24]:
df.head(50)

,Location,Area,Bed,Bath,Price,location,area_sqft,avg_rent
0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand,bashundhara r-a,1600,20000.0
1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand,tejgaon,900,20000.0
2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand,mohammadpur,1250,18000.0
3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,75 Thousand,gulshan,2200,75000.0
4,"Baridhara, Dhaka","2,200 sqft",3,3,75 Thousand,baridhara,2200,75000.0
5,"Bashundhara R-A, Dhaka","3,000 sqft",4,5,50 Thousand,bashundhara r-a,3000,50000.0
6,"Baridhara, Dhaka","2,300 sqft",3,3,75 Thousand,baridhara,2300,75000.0
7,"PC Culture Housing, Mohammadpur, Dhaka",950 sqft,2,2,14 Thousand,mohammadpur,950,14000.0
8,"Jigatola, Hazaribag, Dhaka","1,600 sqft",3,3,28 Thousand,hazaribag,1600,28000.0
9,"West Kazipara, Mirpur, Dhaka","1,150 sqft",3,3,19 Thousand,mirpur,1150,19000.0


In [25]:
df['Price'].isnull().sum()

0

In [26]:
df['avg_rent'].isnull().sum()

0

### Rename Bed and Bath Column

In [27]:
df.rename(columns = {'Bed': 'bedroom', 'Bath': 'bathroom'}, inplace=True)

In [28]:
df.head()

,Location,Area,bedroom,bathroom,Price,location,area_sqft,avg_rent
0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand,bashundhara r-a,1600,20000.0
1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand,tejgaon,900,20000.0
2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand,mohammadpur,1250,18000.0
3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,75 Thousand,gulshan,2200,75000.0
4,"Baridhara, Dhaka","2,200 sqft",3,3,75 Thousand,baridhara,2200,75000.0


## Final Dataframe

In [29]:
df_preprocessed = df[['area_sqft', 'bedroom', 'bathroom', 'location', 'avg_rent']]

In [30]:
df_preprocessed.head()

,area_sqft,bedroom,bathroom,location,avg_rent
0,1600,3,3,bashundhara r-a,20000.0
1,900,2,2,tejgaon,20000.0
2,1250,3,3,mohammadpur,18000.0
3,2200,3,4,gulshan,75000.0
4,2200,3,3,baridhara,75000.0


In [31]:
df_preprocessed.shape

(28800, 5)

In [32]:
df_preprocessed.isnull().sum()

area_sqft     0
bedroom       0
bathroom      0
location     13
avg_rent      0
dtype: int64

In [33]:
df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   area_sqft  28800 non-null  int64  
 1   bedroom    28800 non-null  int64  
 2   bathroom   28800 non-null  int64  
 3   location   28787 non-null  object 
 4   avg_rent   28800 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.1+ MB


In [34]:
df_preprocessed.dropna(axis=0, inplace=True)

In [35]:
df_preprocessed.isnull().sum()

area_sqft    0
bedroom      0
bathroom     0
location     0
avg_rent     0
dtype: int64

In [36]:
df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28787 entries, 0 to 28799
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   area_sqft  28787 non-null  int64  
 1   bedroom    28787 non-null  int64  
 2   bathroom   28787 non-null  int64  
 3   location   28787 non-null  object 
 4   avg_rent   28787 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.3+ MB


## Train Test Split

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
train_df, test_df = train_test_split(df_preprocessed, test_size=0.3, random_state=42)

In [39]:
print(f'train data size: {train_df.shape}')
print(f'test data size: {test_df.shape}')

train data size: (20150, 5)
test data size: (8637, 5)


## Save processed DataFrame

In [40]:
df_preprocessed.to_csv('data_ingestion/processed_data.csv', index=False)
train_df.to_csv('data_ingestion/train_df.csv', index=False)
test_df.to_csv('data_ingestion/test_df.csv', index=False)

### Check loaded or not successfully

In [41]:
new_df = pd.read_csv('data_ingestion/processed_data.csv')
train_df = pd.read_csv('data_ingestion/train_df.csv')
test_df = pd.read_csv('data_ingestion/test_df.csv')

In [42]:
print(new_df.shape)
print(train_df.shape)
print(test_df.shape)

(28787, 5)
(20150, 5)
(8637, 5)


In [43]:
new_df.head()

,area_sqft,bedroom,bathroom,location,avg_rent
0,1600,3,3,bashundhara r-a,20000.0
1,900,2,2,tejgaon,20000.0
2,1250,3,3,mohammadpur,18000.0
3,2200,3,4,gulshan,75000.0
4,2200,3,3,baridhara,75000.0


In [44]:
train_df.head(3)

,area_sqft,bedroom,bathroom,location,avg_rent
0,900,2,2,uttara,18000.0
1,1200,3,3,shahjahanpur,30000.0
2,720,2,2,mohammadpur,17000.0


In [45]:
test_df.head(3)

,area_sqft,bedroom,bathroom,location,avg_rent
0,700,2,2,badda,16000.0
1,900,2,2,uttara,17000.0
2,700,2,2,mirpur,15000.0


# Fine!  Everythin is okay